In [1]:
from utils.prompt_fetch import *
from utils.agent import Agent
from utils.general import *

In [2]:
# get all the files in data folder
files_to_process =  get_files_in_directory("../data/")

In [3]:
files_to_process

['../data/text1.txt']

In [4]:
# get prompts 
prompts = get_prompts("prompts.yaml")

In [5]:
prompts

{'system_prompt_inferential': '[ROLE]:\nYou are an experienced college instructor. You are an expert in writing multiple-choice questions to assess students’ understanding of academic texts (e.g., chapters in textbooks, academic articles).\n\n[TASK]:\nYou are asked to write an inferential multiple-choice question based on an academic text provided by the user. \nFollow the guidelines in [GUIDELINES] to do exactly what you are instructed to do. \n\n[DEFINITION]\n**What are inferential questions?**\nInferential questions require students to go beyond the explicit information in a text and draw conclusions based on evidence, reasoning, and prior knowledge. Unlike literal questions, which ask for directly stated facts, inferential questions challenge students to interpret, analyze, or predict based on textual clues.\nFor example, \nExcerpt:\n"Studies have shown that prolonged exposure to stress can negatively impact cognitive function, particularly memory and decision-making. This is becau

In [6]:
text = read_text_file(files_to_process[0])

In [7]:
text

'Energy and trophic levels: Ecological pyramids How can you show how energy is used in an ecosystem? Ecologists use food chains and food webs to model the distribution of matter and energy within an ecosystem. They also use another kind of model, called an ecological pyramid. An ecological pyramid shows how energy flows through an ecosystem. The base of the ecological pyramid represents the autotrophs, or first trophic level. Higher trophic levels are layered on top of one another. The initial source of energy for all ecological pyramids is energy from the sun. Energy decreases at each succeeding trophic level. The total energy transfer from one trophic level to the next is only about ten percent because organisms fail to capture and eat all the food available at the trophic level below them. When an organism consumes food, it uses some of the energy in the food for metabolism, some for building body tissues, and some is given off as waste. When the organism is eaten, the energy that w

In [8]:

system_prompt_inferential = prompts["system_prompt_inferential"]
user_prompt = prompts["user_prompt"].format(text=text)

In [9]:
model = "gpt-3.5-turbo"
system_prompt = system_prompt_inferential
user_prompt = user_prompt


In [10]:
test_agent = Agent(model=model, system_prompt=system_prompt, user_prompt=user_prompt)

In [12]:
test_agent.completion_generation()

'Based on the information in the text, what is the likely reason for the decrease in energy at each succeeding trophic level in the ecosystem model?\n\nA) Organisms become more efficient at capturing and consuming food at higher trophic levels.\nB) Autotrophs transfer less energy to higher trophic levels due to competition for resources.\nC) Organisms at higher trophic levels use more energy for metabolism and building body tissues.\nD) Herbivores at the lower trophic levels consume too much energy, leaving insufficient for higher trophic levels.\nE) Energy from the sun is absorbed faster at lower trophic levels, leading to reduced availability at higher levels.\n\nCorrect Answer: C) Organisms at higher trophic levels use more energy for metabolism and building body tissues.'

In [13]:
test_agent.get_metadata()

{'task': None,
 'system_prompt': '[ROLE]:\nYou are an experienced college instructor. You are an expert in writing multiple-choice questions to assess students’ understanding of academic texts (e.g., chapters in textbooks, academic articles).\n\n[TASK]:\nYou are asked to write an inferential multiple-choice question based on an academic text provided by the user. \nFollow the guidelines in [GUIDELINES] to do exactly what you are instructed to do. \n\n[DEFINITION]\n**What are inferential questions?**\nInferential questions require students to go beyond the explicit information in a text and draw conclusions based on evidence, reasoning, and prior knowledge. Unlike literal questions, which ask for directly stated facts, inferential questions challenge students to interpret, analyze, or predict based on textual clues.\nFor example, \nExcerpt:\n"Studies have shown that prolonged exposure to stress can negatively impact cognitive function, particularly memory and decision-making. This is be